In [ ]:
!conda install scikit-learn
!conda update scikit-learn
!conda install joblib 
!conda update joblib 

In [1]:
 import pandas as pd

In [3]:
df = pd.read_csv("exoplanet_data.csv")
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [4]:
space = df['koi_disposition'] == "FALSE POSITIVE"
df.loc[space, "koi_disposition"] = "False_positive"
df['koi_disposition']

0            CONFIRMED
1       False_positive
2       False_positive
3            CONFIRMED
4            CONFIRMED
             ...      
6986    False_positive
6987    False_positive
6988         CANDIDATE
6989    False_positive
6990    False_positive
Name: koi_disposition, Length: 6991, dtype: object

In [7]:
from sklearn.model_selection import train_test_split
y = df["koi_disposition"]
x = df.drop(columns=["koi_disposition"])

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=2, stratify=y)
x_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
5812,0,0,1,0,1.000013,3.960000e-06,-3.960000e-06,131.761400,0.003800,-0.003800,...,-103,3.534,0.224,-0.096,3.605,0.595,-1.189,298.47495,39.923210,11.942
4810,0,1,0,0,27.659608,5.840000e-07,-5.840000e-07,141.247888,0.000017,-0.000017,...,-143,4.593,0.035,-0.105,0.768,0.132,-0.066,286.81644,46.884369,14.706
1815,0,0,0,0,6.124798,3.240000e-05,-3.240000e-05,134.938030,0.004280,-0.004280,...,-133,4.285,0.137,-0.112,1.202,0.208,-0.189,288.37024,43.354610,13.268
6288,0,0,1,1,5.243978,2.932000e-04,-2.932000e-04,132.682900,0.055200,-0.055200,...,-230,4.481,0.054,-0.216,0.956,0.306,-0.102,291.82001,41.697842,14.967
5786,0,0,1,1,0.706430,4.930000e-06,-4.930000e-06,132.142080,0.006940,-0.006940,...,-167,4.689,0.052,-0.024,0.568,0.030,-0.034,290.59076,44.002579,14.001


In [9]:
from sklearn.preprocessing import MinMaxScaler
x_scaler = MinMaxScaler().fit(x_train)

x_train_scaled = x_scaler.transform(x_train)
x_test_scaled = x_scaler.transform(x_test)

In [10]:
 import tensorflow as tf
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [11]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
y_train_encoded = label_encoder.transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

y_train_categorical = to_categorical(y_train_encoded)
y_test_categorical = to_categorical(y_test_encoded)

In [12]:
y_train_categorical.shape

(5243, 3)

In [13]:
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=40))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

In [14]:
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [15]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               4100      
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 303       
Total params: 14,503
Trainable params: 14,503
Non-trainable params: 0
_________________________________________________________________


In [16]:
callbacks = [EarlyStopping(monitor='val_loss', patience=2)]
model.fit(x_train_scaled,
    y_train_categorical,
    callbacks=callbacks,
    epochs=60,
    shuffle=True,
    verbose=2)

ERROR! Session/line number was not unique in database. History logging moved to new session 314
Epoch 1/60
 - 1s - loss: 0.5955 - accuracy: 0.7049
Epoch 2/60


C:\Users\matth\Anaconda3\envs\PythonAdv\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


 - 0s - loss: 0.3767 - accuracy: 0.8110
Epoch 3/60
 - 0s - loss: 0.3596 - accuracy: 0.8198
Epoch 4/60
 - 0s - loss: 0.3513 - accuracy: 0.8230
Epoch 5/60
 - 0s - loss: 0.3442 - accuracy: 0.8314
Epoch 6/60
 - 0s - loss: 0.3362 - accuracy: 0.8364
Epoch 7/60
 - 0s - loss: 0.3341 - accuracy: 0.8344
Epoch 8/60
 - 0s - loss: 0.3317 - accuracy: 0.8409
Epoch 9/60
 - 0s - loss: 0.3287 - accuracy: 0.8472
Epoch 10/60
 - 0s - loss: 0.3251 - accuracy: 0.8484
Epoch 11/60
 - 0s - loss: 0.3211 - accuracy: 0.8491
Epoch 12/60
 - 0s - loss: 0.3277 - accuracy: 0.8476
Epoch 13/60
 - 0s - loss: 0.3173 - accuracy: 0.8497
Epoch 14/60
 - 0s - loss: 0.3081 - accuracy: 0.8592
Epoch 15/60
 - 0s - loss: 0.3020 - accuracy: 0.8606
Epoch 16/60
 - 0s - loss: 0.3057 - accuracy: 0.8585
Epoch 17/60
 - 0s - loss: 0.2989 - accuracy: 0.8642
Epoch 18/60
 - 0s - loss: 0.3048 - accuracy: 0.8644
Epoch 19/60
 - 0s - loss: 0.2994 - accuracy: 0.8634
Epoch 20/60
 - 0s - loss: 0.2955 - accuracy: 0.8642
Epoch 21/60
 - 0s - loss: 0.291

In [17]:
model_loss, model_accuracy = model.evaluate(
    x_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Normal Neural Network - Loss: 0.24630453459730955, Accuracy: 0.9078947305679321


In [18]:
encoded_predictions = model.predict_classes(x_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [19]:
 print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

Predicted classes: ['False_positive' 'False_positive' 'False_positive' 'False_positive'
 'CONFIRMED']
Actual Labels: ['False_positive', 'False_positive', 'False_positive', 'False_positive', 'CANDIDATE']


In [20]:
import joblib
filename = 'deep_learning.sav'
joblib.dump(model, filename)

['deep_learning.sav']